## Funciones de utilidad

In [17]:
import sympy as sp
from IPython.display import display, Math

def print_math(sympy_exp, pre_symbol = '', post_symbol = '', convert_pre_post = False):
    # Map some symbols to be reaplaced before printing to keep desired order
    rep_map = {
        'zt': 't',
        'zU_p': 'U_p',
        'z\\dot{U}_p': '\\dot{U}_p',
        'z\\ddot{U}_p': '\\ddot{U}_p',
        'zU': 'U',
        'z\\dot{U}': '\\dot{U}',
        'z\\ddot{U}': '\\ddot{U}',
        'a\\xi': '\\xi',
        'am': 'm',
        'bc': 'c',
    }
    latex_str = sp.latex(sympy_exp)
    if convert_pre_post:
        pre_symbol_latex = sp.latex(pre_symbol)
        post_symbol_latex = sp.latex(post_symbol)
    for old_str in rep_map:
        latex_str = str(latex_str).replace(old_str, rep_map[old_str])
        if pre_symbol != '' and convert_pre_post:
            pre_symbol_latex = pre_symbol_latex.replace(old_str, rep_map[old_str])
        if post_symbol != '' and convert_pre_post:
            post_symbol_latex = post_symbol_latex.replace(old_str, rep_map[old_str])
            
    # if pre_symbol != '' and convert_pre_post:
    #     pre_symbol = pre_symbol_latex
    # if post_symbol != '' and convert_pre_post:
    #     post_symbol = post_symbol_latex
        
    if pre_symbol != '':
        if convert_pre_post:
            pre_symbol = pre_symbol_latex
        latex_str = pre_symbol + ' = ' + latex_str
    if post_symbol != '':
        if convert_pre_post:
            post_symbol = post_symbol_latex
        latex_str = latex_str + ' = ' + post_symbol
    # print(latex_str)
    display(Math(f'{latex_str} \n'))

def print_text(text_str):
    display(Math(f'\\textrm{{{text_str}}}'))

## Encontrando la solución particular

In [18]:
# General symbols
Uu, Uv, Ua, C1h, C2h, wn, wd, t, m, k, to, z, c, r, ts = sp.symbols('zU, z\\dot{U}, z\\ddot{U}, C_{1h}, C_{2h}, w_n, w_d, zt, am, k, zt_0, a\\xi, bc, r, tr')

# Symbols for the particular solution
Uup, Uvp, Uap, C1p, C2p, Uest = sp.symbols('zU_p, z\\dot{U}_p, z\\ddot{U}_p, C_{1p}, C_{2p}, U_{est}')

# Symbols specific for the loading function
Ome, Fo, a_fac = sp.symbols('\Omega, F_0, a_{fac}')

get_uest = True

# Parts of the particular solution and external force -- Solve with an offset variable
# Sine accel
# part_factors = [sp.cos(Ome * t), sp.sin(Ome * t)]
# part_cts = [C1p, C2p]
# ext_force_orig = Fo * sp.sin(Ome*t)
# ext_force = ext_force_orig

# Cosine accel
part_factors = [sp.cos(Ome * t), sp.sin(Ome * t)]
part_cts = [C1p, C2p]
ext_force_orig = Fo * sp.cos(Ome*t)
ext_force = ext_force_orig

# # Free vibration
# part_factors = [1, 1]
# part_cts = [C1p, C2p]
# ext_force_orig = 0
# ext_force = ext_force_orig

# Constant
# part_factors = [t, 1]
# part_cts = [C2p, C1p]
# ext_force_orig = Fo
# ext_force = ext_force_orig

# Linear acceleration (negative slope)
# part_factors = [t, 1]
# part_cts = [C2p, C1p]
# ext_force_orig = Fo * (1 - t/(ts/2))
# ext_force = ext_force_orig

# Linear acceleration (negative slope - With offset)
# part_factors = [t, 1]
# part_cts = [C2p, C1p]
# ext_force_orig = Fo * (1 - t/(ts/2)) + 0.5
# ext_force_orig = Fo * (1.8 - 2.6*t/1.8)
# ext_force_orig = Fo * (-0.6 + t)
# ext_force = ext_force_orig
# get_uest = True

# Linear acceleration (positive slope)
# part_factors = [t, 1]
# part_cts = [C2p, C1p]
# ext_force_orig = -Fo * (1 - t/(ts/2))
# ext_force = ext_force_orig

if ext_force != 0:
    # Diferential equation
    print_text('Ecuacion diferencial de gobierno: ')
    ed = m * Ua + c * Uv + k * Uu # - Fo * sp.sin(Ome * t)
    print_math(ed, '', ext_force, True)

    # TODO Divide by m to make things easier and get \xi and wn into play
    ed = sp.expand(ed / m)
    ed = ed.subs(c, 2*z*wn*m)
    ed = ed.subs(k/m, wn**2)
    ext_force = ext_force / m
    print_math(ed, '', ext_force, True)

    # Solution's shape for displacement, velocity and acceleration
    print_text('Forma de la solucion: ')
    Ups = (part_cts[0] * part_factors[0] + part_cts[1] * part_factors[1])
    print_math(Ups, 'U_p',)
    Vps = sp.diff(Ups, t)
    print_math(Vps, '\\dot{U_p}')
    Aps = sp.diff(Vps, t)
    print_math(Aps, '\\ddot{U_p}')

    # Plug solution function into the governing differential equation
    print_text('Reemplazando en la ecuacion diferencial: ')
    ed_rep = ed.subs({Uu: Ups, Uv: Vps, Ua: Aps})
    print_math(ed_rep, '', ext_force, True)

    # TODO Introduce the factor r = Ome/wn
    print_text('Introduciendo factores de simplificacion: ')
    if get_uest:
        ed_rep = sp.expand(ed_rep / wn**2)
        ed_rep = ed_rep.subs(Ome / wn, r)
        ext_force = sp.expand(ext_force / wn**2)
        ext_force = ext_force.subs(m * wn**2, k)
        ext_force = ext_force.subs(Fo / k, Uest)
    print_math(ed_rep, '', ext_force, True)

    # Get the factors for each part of the function to solve it
    factors_coeff = sp.reduced(ed_rep, part_factors)
    factors_coeff_ext = sp.reduced(ext_force, part_factors)

    # Create the system of equations
    eqns = []
    for i in range(len(factors_coeff[0])):
        eqns.append(factors_coeff[0][i] - factors_coeff_ext[0][i])

    # Solve the system of equations
    sol = sp.solve(eqns, part_cts)

    for cnt in part_cts:
        if not cnt in sol:
            sol[cnt] = 0

    print_text('Coeficientes encontrados: ')
    for res in sol:
        sol[res] = sp.simplify(sp.factor(sol[res]))
        print_math(sol[res], res, '', True)
else:
    Ups = 0
    Vps = 0
    Aps = 0
    C1p = 0
    C2p = 0

print_math(Ups, 'U_p',)
print_math(Vps, '\\dot{U}_p',)
print_math(Aps, '\\ddot{U}_p',)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Solución total con $U = U_p + U_h$

In [19]:
Uuo, Uvo, Uao = sp.symbols('zU_0, \\dot{U}_0, \\ddot{U}_0')

# Diferential equation
print_text('Ecuacion diferencial de gobierno: ')
ed = m * Ua + c * Uv + k * Uu
print_math(ed, '', '0')
print_text('Forma de la solucion: ')
Us = (C1h * sp.cos(wd * t) + C2h * sp.sin(wd * t)) * sp.exp(-z*wn*t) + Ups
print_math(Us, 'U')
Vs = sp.diff(Us, t)
print_math(Vs, '\\dot{U}')
As = sp.diff(Vs, t)
print_math(As, '\\ddot{U}')

print_text('Dadas las condiciones de borde, encontrar las condiciones de borde: ')
Uso = Us - Uuo
Uso = Uso.subs(t, to)
print_math(Uso, '', '0')
Uso = Uso.subs(to, 0)
Vso = Vs - Uvo
Vso = Vso.subs(t, to)
print_math(Vso, '', '0')
Vso = Vso.subs(to, 0)

print_text('Sistema de ecuaciones a resolver: ')
print_math(Uso, '', '0')
print_math(Vso, '', '0')


print_text('Coeficientes encontrados: ')
coeff_sol = sp.solve([Uso, Vso], [C1h, C2h])
for coeff in coeff_sol:
    coeff_sol[coeff] = sp.simplify(sp.factor(coeff_sol[coeff]))
    print_math(coeff_sol[coeff], sp.latex(coeff))

print_text('Soluciones finales:')
print_math(Us, 'U')
print_math(Vs, '\\dot{U}')
print_math(As, '\\ddot{U}')

print_text('Solucion homogenea:')
Uh = (C1h * sp.cos(wd * t) + C2h * sp.sin(wd * t)) * sp.exp(-z*wn*t)
print_math(Uh, 'U')
Vh = sp.diff(Uh, t)
print_math(Vh, '\\dot{U}')
Ah = sp.diff(Vh, t)
print_math(Ah, '\\ddot{U}')


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Traduciendo las expresiones a Matlab

In [20]:
def sympy2matlab(spExpr):
    rep_map = {
        'w_n': 'wn',
        'F_0': 'Fo',
        '\\Omega': 'Ome',
        'zU_0': 'Uo',
        'r': 'r',
        'w_d': 'wd',
        'zt': 't',
        'U_{est}': 'Uest',
        '\\dot{U}_0': 'Vo',
        'a\\xi': 'z',
        'C_{1h}': 'C1h',
        'C_{2h}': 'C2h',
        'C_{1p}': 'C1p',
        'C_{2p}': 'C2p',
        'tr': 'tr',
        'am': 'me',
    }
    str_expr = str(spExpr).replace('**', '^')
    str_expr = str_expr.replace('*exp', '.*exp')
    try:
      used_symbols = spExpr.atoms(sp.Symbol)
    except:
      used_symbols = []
    
    for sym in used_symbols:
        str_expr = str_expr.replace(str(sym), rep_map[str(sym)])
    return str_expr


Up_str = sympy2matlab(Ups)
Vp_str = sympy2matlab(Vps)
Ap_str = sympy2matlab(Aps)

Uh_str = sympy2matlab(Uh)
Vh_str = sympy2matlab(Vh)
Ah_str = sympy2matlab(Ah)

C1h_str = sympy2matlab(coeff_sol[C1h])
C2h_str = sympy2matlab(coeff_sol[C2h])
if C1p != 0:
  C1p_str = sympy2matlab(sol[C1p])
else:
  C1p_str = '0'
if C2p != 0:
  C2p_str = sympy2matlab(sol[C2p])
else:
  C2p_str = '0'

Fext_str = sympy2matlab(ext_force_orig)

print('    % Fuerza externa')
print(f'    Fext = {Fext_str};')
print('    % Constantes particulares')
print(f'    C1p = {C1p_str};')
print(f'    C2p = {C2p_str};')
print('    % Solucion particular')
print(f'    Up = {Up_str};')
print(f'    Vp = {Vp_str};')
print(f'    Ap = {Ap_str};')
print('    % Constantes homogeneas')
print(f'    C1h = {C1h_str};')
print(f'    C2h = {C2h_str};')
print('    % Solucion homogenena')
print(f'    Uh = {Uh_str};')
print(f'    Vh = {Vh_str};')
print(f'    Ah = {Ah_str};')
print('    % Totales')
print(f'    Ut = Uh + Up;')
print(f'    Vt = Vh + Vp;')
print(f'    At = Ah + Ap;')

    % Fuerza externa
    Fext = Fo*cos(Ome*t);
    % Constantes particulares
    C1p = Uest*(1 - r^2)/(4*z^2*r^2 + r^4 - 2*r^2 + 1);
    C2p = 2*Uest*z*r/(4*z^2*r^2 + r^4 - 2*r^2 + 1);
    % Solucion particular
    Up = C1p*cos(Ome*t) + C2p*sin(Ome*t);
    Vp = -C1p*Ome*sin(Ome*t) + C2p*Ome*cos(Ome*t);
    Ap = -C1p*Ome^2*cos(Ome*t) - C2p*Ome^2*sin(Ome*t);
    % Constantes homogeneas
    C1h = -C1p + Uo;
    C2h = (-C1p*z*wn - C2p*Ome + Vo + z*wn*Uo)/wd;
    % Solucion homogenena
    Uh = (C1h*cos(wd*t) + C2h*sin(wd*t)).*exp(-z*wn*t);
    Vh = -z*wn*(C1h*cos(wd*t) + C2h*sin(wd*t)).*exp(-z*wn*t) + (-C1h*wd*sin(wd*t) + C2h*wd*cos(wd*t)).*exp(-z*wn*t);
    Ah = z^2*wn^2*(C1h*cos(wd*t) + C2h*sin(wd*t)).*exp(-z*wn*t) - 2*z*wn*(-C1h*wd*sin(wd*t) + C2h*wd*cos(wd*t)).*exp(-z*wn*t) + (-C1h*wd^2*cos(wd*t) - C2h*wd^2*sin(wd*t)).*exp(-z*wn*t);
    % Totales
    Ut = Uh + Up;
    Vt = Vh + Vp;
    At = Ah + Ap;
